<a href="https://colab.research.google.com/github/ricky-kiva/dl-deep-tf-cv-advanced/blob/main/1_a1_bboxes_birds_caltech.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Object Localization on Caltech Birds**

Connecting Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)

Import packages

In [1]:
import os
import cv2
import numpy as np
import tensorflow as tf

from matplotlib import pyplot as plt

Function: Dataset to numpy

In [4]:
def dataset_to_numpy(dataset, batch_size=0, N=0):

  take_dataset = dataset.shuffle(1024) # take first '1024' data, put into buffer, & randomly shuffle the data

  if batch_size > 0:
    take_dataset = take_dataset.batch(batch_size) # groups dataset elements into batches of size 'batch_size'

  if N > 0:
    take_dataset = take_dataset.take(N) # take the first `N` element

  print(f"Executing eagerly: {tf.executing_eagerly()}")
  if tf.executing_eagerly():
    ds_images, ds_bboxes = [], []
    for img, bbox in take_dataset:
      ds_images.append(img.numpy())
      ds_bboxes.append(bbox.numpy())

    return (np.array(ds_images, dtype=object),
            np.array(ds_bboxes, dtype=object))